# A benchmark model of dynamic portfolio choice with transaction costs

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import torch
import gpytorch
from scipy.optimize import minimize
from torch.optim import Adam
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, MaternKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from cyipopt import minimize_ipopt


%matplotlib inline
%load_ext autoreload
%autoreload 2


Current issues: Gradient of Ct is NaN. Check with pdf

# Current implementation

In [20]:
import torch
import torch.autograd as autograd
import gpytorch
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, MaternKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from cyipopt import minimize_ipopt
import numpy as np

# Parameters
T = 10  # Time horizon
D = 2  # Number of risky assets
r = 0.03  # Risk-free return in pct.
Rf = np.exp(r)  # Risk-free return
tau = 0.01  # Transaction cost rate
beta = 0.975  # Discount factor
gamma = 3.5  # Risk aversion coefficient

# Risky assets - deterministic
mu = np.array([0.07, 0.07])
variance = 0.2**2
Sigma = np.array([[0.04, 0], [0, 0.04]])

# Define the GPR model with ARD
class GPRegressionModel(ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(MaternKernel(nu=1.5, ard_num_dims=train_x.shape[1]))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Function to train the GPR model
def train_gp_model(train_x, train_y):
    likelihood = GaussianLikelihood()
    model = GPRegressionModel(train_x, train_y, likelihood)
    model.train()
    likelihood.train()

    # Use the Adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

    # "Loss" for GPs - the marginal log likelihood
    mll = ExactMarginalLogLikelihood(likelihood, model)

    training_iterations = 40
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()

    print(f"Trained model on inputs: {train_x}")
    print(f"Trained model on targets: {train_y}")        
    
    return model, likelihood
# Utility function
def utility(ct, gamma):
    if gamma == 1:
        return torch.log(ct)
    else:
        return (ct**(1 - gamma)) / (1 - gamma)

def safe_utility(ct, gamma):
    ct = torch.tensor(ct, dtype=torch.float32)  # Ensure ct is a tensor
    ct = torch.clamp(ct, min=1e-6)  # Prevent log(0) or negative values
    return utility(ct, gamma)

# Normalized bond holdings
def normalized_bond_holdings(xt, delta_plus, delta_minus, ct, tau):
    bt = 1 - torch.sum(xt - delta_plus + delta_minus + tau * (delta_plus + delta_minus)) - ct
    return bt

def normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf):
    pi_t1 = bt * Rf + torch.sum((xt + delta_plus - delta_minus) * Rt)
    xt1 = ((xt + delta_plus - delta_minus) * Rt) / pi_t1
    return pi_t1, xt1

def bellman_equation(vt_next_in, vt_next_out, xt, ct, delta_plus, delta_minus, beta, gamma, tau, Rf):
    bt = normalized_bond_holdings(torch.tensor(xt, requires_grad=True), torch.tensor(delta_plus, requires_grad=True), torch.tensor(delta_minus, requires_grad=True), torch.tensor(ct, requires_grad=True), tau)
    Rt = torch.tensor(mu + np.random.multivariate_normal(np.zeros(D), Sigma))  # Simulated return
    pi_t1, xt1 = normalized_state_dynamics(xt, delta_plus, delta_minus, Rt, bt, Rf)
    
    u = safe_utility(ct, gamma)
    xt1_tensor = torch.tensor(xt1, dtype=torch.float32, requires_grad=True)
    
    if vt_next_in is None or vt_next_out is None:
        raise ValueError("vt_next_in or vt_next_out is None")
    
    if is_in_ntr(xt1):
        vt_next_val = vt_next_in(xt1_tensor.unsqueeze(0)).mean()
    else:
        vt_next_val = vt_next_out(xt1_tensor.unsqueeze(0)).mean()
    
    vt = u + beta * torch.mean(pi_t1 ** (1 - gamma) * vt_next_val)
    
    return vt

def solve_optimization(xt, vt_next_in, vt_next_out):
    def objective(params):
        delta_plus = torch.tensor(params[:D], dtype=torch.float32, requires_grad=True)
        delta_minus = torch.tensor(params[D:2*D], dtype=torch.float32, requires_grad=True)
        ct = torch.tensor(params[2*D], dtype=torch.float32, requires_grad=True)
        vt = bellman_equation(vt_next_in, vt_next_out, xt, ct, delta_plus, delta_minus, beta, gamma, tau, Rf)
        return vt

    def gradient(params):
        delta_plus = torch.tensor(params[:D], dtype=torch.float32, requires_grad=True)
        delta_minus = torch.tensor(params[D:2*D], dtype=torch.float32, requires_grad=True)
        ct = torch.tensor(params[2*D], dtype=torch.float32, requires_grad=True)
        vt = bellman_equation(vt_next_in, vt_next_out, xt, ct, delta_plus, delta_minus, beta, gamma, tau, Rf)
        vt.backward()
        
        grad = np.concatenate([
            delta_plus.grad.detach().numpy(),
            delta_minus.grad.detach().numpy(),
            [ct.grad.detach().numpy()]
        ])
        
        return grad

    def constraints(params):
        delta_plus = torch.tensor(params[:D], dtype=torch.float32, requires_grad=True)
        delta_minus = torch.tensor(params[D:2*D], dtype=torch.float32, requires_grad=True)
        ct = torch.tensor(params[2*D], dtype=torch.float32, requires_grad=True)
        no_shorting_plus = delta_plus  # No shorting constraint
        no_shorting_minus = delta_minus  # No shorting constraint
        no_borrowing = normalized_bond_holdings(xt, delta_plus, delta_minus, ct, tau).detach()  # No borrowing constraint
        return torch.cat([no_shorting_plus, no_shorting_minus, torch.tensor([no_borrowing])]).detach().numpy()

    def jacobian(params):
        delta_plus = torch.tensor(params[:D], dtype=torch.float32, requires_grad=True)
        delta_minus = torch.tensor(params[D:2*D], dtype=torch.float32, requires_grad=True)
        ct = torch.tensor(params[2*D], dtype=torch.float32, requires_grad=True)
        bt = normalized_bond_holdings(xt, delta_plus, delta_minus, ct, tau)
        grads = autograd.grad(bt, [delta_plus, delta_minus, ct], create_graph=True)
        jac = np.concatenate([g.detach().numpy().flatten() for g in grads])
        return jac

    initial_guesses = [np.zeros(2*D + 1) for _ in range(5)]
    bounds = [(0, 1)] * (2*D + 1)
    constraints_def = [{'type': 'ineq', 'fun': lambda x: constraints(x)}]
    tol = 1e-6

    for initial_guess in initial_guesses:
        result = minimize_ipopt(objective, initial_guess, bounds=bounds, constraints=constraints_def, jac=gradient, options={'tol': tol, 'maxiter': 300})
        if result.success:
            break
    
    delta_plus = result.x[:D]
    delta_minus = result.x[D:2*D]
    ct = result.x[2*D]
    return delta_plus, delta_minus, ct


def dynamic_programming(T, N, D, gamma, beta, tau, Rf):
    V = initialize_value_function(T, gamma)
    
    for t in range(T-1, -1, -1):
        Xt = sample_state_points(D)
        
        vt_values_in = []
        vt_values_out = []
        policies_in = []
        policies_out = []
        
        for xt in Xt:
            print(f"Time step {t}, state {xt}")
            if V[t+1][0] is None or V[t+1][1] is None:
                print(f"V[t+1][0] or V[t+1][1] is None at time {t+1}")
            
            delta_plus, delta_minus, ct = solve_optimization(xt, V[t+1][0], V[t+1][1])
            vt_value = bellman_equation(V[t+1][0], V[t+1][1], xt, ct, delta_plus, delta_minus, beta, gamma, tau, Rf).item()
            
            if is_in_ntr(xt):
                vt_values_in.append(vt_value)
                policies_in.append((xt, delta_plus, delta_minus, ct))
                # Placeholder!
                vt_values_out.append(vt_value)
                policies_out.append((xt, delta_plus, delta_minus, ct))
            else:
                vt_values_out.append(vt_value)
                policies_out.append((xt, delta_plus, delta_minus, ct))
        
        Xt_tensor_in = torch.tensor([x[0].numpy() for x in policies_in], dtype=torch.float32)
        vt_values_tensor_in = torch.tensor(vt_values_in, dtype=torch.float32)
        
        V[t][0], _ = train_gp_model(Xt_tensor_in, vt_values_tensor_in)

        Xt_tensor_out = torch.tensor([x[0].numpy() for x in policies_out], dtype=torch.float32)
        vt_values_tensor_out = torch.tensor(vt_values_out, dtype=torch.float32)
        
        V[t][1], _ = train_gp_model(Xt_tensor_out, vt_values_tensor_out)
    
    return V  

def is_in_ntr(xt):
    # Placeholder for logic to determine if a point is inside the NTR
    # This will use the approximation method described in the PDFs
    return True  # Change this based on actual logic

def initialize_value_function(T, gamma):
    V = [[None, None] for _ in range(T + 1)]
    def V_terminal(xT):
        return utility(1 - tau * torch.sum(torch.abs(xT)), gamma)
    V[T][0] = V[T][1] = lambda x: V_terminal(x)
    return V

# Sample state points function
def sample_state_points(D):
    points = []
    for i in range(2 ** D):
        point = [(i >> j) & 1 for j in range(D)]
        points.append(point)
    points.append([0] * D)
    for i in range(1, 2 ** D):
        for j in range(i):
            midpoint = [(a + b) / 2 for a, b in zip(points[i], points[j])]
            points.append(midpoint)
    return torch.tensor(points, dtype=torch.float32)

# Define parameters and run the algorithm
N = 50  # Number of sample points
V = dynamic_programming(T, N, D, gamma, beta, tau, Rf)

# V now contains the approximated value functions for each time period

Time step 9, state tensor([0., 0.])


/var/folders/hz/t94d7ym95fx1sf3fpl6b9bcm0000gn/T/ipykernel_53472/1411705395.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bt = normalized_bond_holdings(torch.tensor(xt, requires_grad=True), torch.tensor(delta_plus, requires_grad=True), torch.tensor(delta_minus, requires_grad=True), torch.tensor(ct, requires_grad=True), tau)
/var/folders/hz/t94d7ym95fx1sf3fpl6b9bcm0000gn/T/ipykernel_53472/1411705395.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ct = torch.tensor(ct, dtype=torch.float32)  # Ensure ct is a tensor
/var/folders/hz/t94d7ym95fx1sf3fpl6b9bcm0000gn/T/ipykernel_53472/1411705395.py:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor

AttributeError: 'NoneType' object has no attribute 'detach'